In [1]:
import sys
sys.path.append('..')

In [2]:
from utils import io

In [21]:
import numpy as np
import torch
import dgl
import os.path as osp
from tqdm import tqdm
from copy import deepcopy

In [4]:
g = io.load_pickle('/home/sxr/data/Amazon_CSJ/raw_graph.dgl.pkl')
g

Graph(num_nodes=15164975, num_edges=32292099,
      ndata_schemes={}
      edata_schemes={})

In [5]:
in_degrees = g.in_degrees()
out_degrees = g.out_degrees()

In [7]:
(in_degrees > 0).sum()

tensor(12483678)

In [8]:
(out_degrees > 0).sum()

tensor(2681297)

In [10]:
((in_degrees > 0).sum() + (out_degrees > 0).sum()) == g.num_nodes()

tensor(True)

In [12]:
all_node_id = g.nodes()

In [11]:
user_flag = out_degrees > 0
g.ndata['user_flag'] = user_flag

In [208]:
def save_edge_txt(g, file):
    E_src, E_dst = g.edges()
    num_users = g.ndata['user_flag'].sum().item()
    E_src = E_src.numpy()
    E_dst = E_dst.numpy() - num_users
    E = np.stack([E_src, E_dst]).transpose()
    np.savetxt(file, E, fmt='%d')

In [ ]:
np.random.seed(2022)

In [421]:
all_node_id = g.nodes()
in_degrees = g.in_degrees()
out_degrees = g.out_degrees()

In [422]:
all_item_id = all_node_id[~g.ndata['user_flag']]
len(all_item_id)

3397863

In [423]:
all_item_id

tensor([1035512, 1035513, 1035514,  ..., 4433372, 4433373, 4433374])

In [424]:
all_user_id = all_node_id[g.ndata['user_flag']]
len(all_user_id)

1035512

In [425]:
all_user_id

tensor([      0,       1,       2,  ..., 1035509, 1035510, 1035511])

In [426]:
# d_items = in_degrees[all_item_id]

In [427]:
# np.percentile(d_items, 99.99)

In [428]:
rm_item_id = all_item_id[in_degrees[all_item_id] < 2]
len(rm_item_id) / len(all_item_id)

0.0

In [429]:
# np.random.shuffle(rm_item_id.numpy())

In [430]:
# rm_item_id = rm_item_id[:int(0.05 * len(all_item_id))]

In [431]:
# d_users = out_degrees[all_user_id]

In [432]:
# np.percentile(d_users, 99.5)

In [433]:
rm_user_id = all_user_id[out_degrees[all_user_id] < 2]
# rm_user_id = all_user_id[out_degrees[all_user_id] > 500]
len(rm_user_id) / len(all_user_id)

0.0

In [434]:
# np.random.shuffle(rm_user_id.numpy())
# rm_user_id = rm_user_id[:int(0.05 * len(all_user_id))]

In [435]:
rm_nodes = torch.cat([rm_item_id, rm_user_id])
len(rm_nodes) / len(all_node_id)

0.0

In [436]:
_g = dgl.remove_nodes(g, rm_nodes)
_g

Graph(num_nodes=4433375, num_edges=15318750,
      ndata_schemes={'user_flag': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

In [437]:
g = _g

In [438]:
len(rm_user_id)

0

In [439]:
len(rm_item_id)

0

In [440]:
save_edge_txt(g, '/home/sxr/data/social_and_user_item/datasets/instance_amazon-csj-4.4m/train.txt')